In [22]:
#Cosine Similarity
corpus = [
    "This is a sample document.",
    "Here is another document.",
    "And this is a third document."
]

query = "This is a query text"

#TF-IDF
first_sentence = "The sky is blue and beautiful"
second_sentence = "Love is in the air"

In [23]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = CountVectorizer()
matrix = vectorizer.fit_transform(corpus)

# print(vectorizer.transform(corpus))
# print(matrix.toarray())

query_v = vectorizer.transform([query])
similarities = cosine_similarity(query_v, matrix)
data = {'Document' : corpus, "similarities" : similarities[0]}
df = pd.DataFrame(data)
df

,Document,similarities
0,This is a sample document.,0.707107
1,Here is another document.,0.353553
2,And this is a third document.,0.632456


In [24]:
#TF-DF
import nltk
nltk.download("punkt")
from nltk.tokenize import word_tokenize
first_sentence = word_tokenize(first_sentence.lower())
second_sentence = word_tokenize(second_sentence.lower())
word_set = set(first_sentence).union(set(second_sentence))
print(word_set)

word_dict_a = dict.fromkeys(word_set,0)
word_dict_b = dict.fromkeys(word_set, 0)

for word in first_sentence:
    word_dict_a[word] += 1

for word in second_sentence:
    word_dict_b[word] += 1

pd.DataFrame([word_dict_a, word_dict_b])

{'the', 'and', 'is', 'beautiful', 'sky', 'air', 'in', 'love', 'blue'}


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,the,and,is,beautiful,sky,air,in,love,blue
0,1,1,1,1,1,0,0,0,1
1,1,0,1,0,0,1,1,1,0


In [25]:
def computeTF(word_dict, doc):
    tf_dict = {}
    length = len(doc)
    
    for word, count in word_dict.items():
        tf_dict[word] = count/length
        
    return(tf_dict)

tf_first = computeTF(word_dict_a, first_sentence)
tf_second = computeTF(word_dict_b, second_sentence)
tf = pd.DataFrame([tf_first, tf_second])
tf

,the,and,is,beautiful,sky,air,in,love,blue
0,0.166667,0.166667,0.166667,0.166667,0.166667,0.0,0.0,0.0,0.166667
1,0.200000,0.000000,0.200000,0.000000,0.000000,0.2,0.2,0.2,0.000000


In [27]:
#Function buat compute IDF (Inverse document frequency)
import math
def computeIDF(doc_list):
    idf_dict = {}
    count = len(doc_list)
    
    all_words = set()
    for d in doc_list:
        all_words.update(d.keys())
    
    idf_dict = dict.fromkeys(all_words, 0)
    
    for word in all_words:
        for d in doc_list:
            if d[word] > 0:
                idf_dict[word] += 1
    
    for word, value in idf_dict.items():
        idf_dict[word] = math.log10(count/value)
    
    idf_df = pd.DataFrame(list(idf_dict.items()), columns=["word", "IDF"])
    return idf_df

idf = computeIDF([word_dict_a, word_dict_b])
idf


,word,IDF
0,the,0.00000
1,and,0.30103
2,is,0.00000
3,beautiful,0.30103
4,blue,0.30103
5,sky,0.30103
6,air,0.30103
7,in,0.30103
8,love,0.30103


In [28]:
#function buat compute TF-IDF

def computeTFIDF(tf, idf):
    tfidf = {}
    
    for word, value in tf.items():
        tfidf[word] = value * idf.loc[idf["word"] == word, "IDF"].values[0]
    
    return tfidf

tf_idf_first = computeTFIDF(tf_first, idf)
tf_idf_second = computeTFIDF(tf_second, idf)

tf_idf = pd.DataFrame([tf_idf_first, tf_idf_second])
tf_idf

,the,and,is,beautiful,sky,air,in,love,blue
0,0.0,0.050172,0.0,0.050172,0.050172,0.000000,0.000000,0.000000,0.050172
1,0.0,0.000000,0.0,0.000000,0.000000,0.060206,0.060206,0.060206,0.000000
